In [44]:
import json
import os
import sys
import requests
from tqdm import tqdm

In [54]:
queries = {
    "kong-access-":{
        "bool": {
            "must": [
                {
                    "range": {
                        "@timestamp": {
                            "gte": "2025-01-02T00:00:00.000Z",
                            "lte": "2025-01-03T00:00:00.000Z",
                            "format": "strict_date_optional_time"
                        }
                    }
                }
            ]
        }
    },
    "metricbeat-": {
        "bool": {
            "must": [
                {
                            "query_string": {
                                "query": "cee25daa-3fd9-441b-af33-8211e3649f3e",
                                "default_operator": "AND"
                            }
                        },
                {
                    "range": {
                        "@timestamp": {
                            "gte": "2025-01-02T00:00:00.000Z",
                            "lte": "2025-01-03T00:00:00.000Z",
                            "format": "strict_date_optional_time"
                        }
                    }
                }
            ]
        }
    },
    "traces-apm":{
        "bool": {
            "must": [
                {
                    "range": {
                        "@timestamp": {
                            "gte": "2025-01-02T00:00:00.000Z",
                            "lte": "2025-01-03T00:00:00.000Z",
                            "format": "strict_date_optional_time"
                        }
                    }
                }
            ]
        }
    },
    "apm-":{
        "bool": {
            "must": [
                {
                    "range": {
                        "@timestamp": {
                            "gte": "2025-01-02T00:00:00.000Z",
                            "lte": "2025-01-03T00:00:00.000Z",
                            "format": "strict_date_optional_time"
                        }
                    }
                }
            ]
        }
    },
    "logs-apm":{
        "bool": {
            "must": [
                {
                    "range": {
                        "@timestamp": {
                            "gte": "2025-01-02T00:00:00.000Z",
                            "lte": "2025-01-03T00:00:00.000Z",
                            "format": "strict_date_optional_time"
                        }
                    }
                }
            ]
        }
    },
    "metrics-apm":{
        "bool": {
            "must": [
                {
                    "range": {
                        "@timestamp": {
                            "gte": "2025-01-02T00:00:00.000Z",
                            "lte": "2025-01-03T00:00:00.000Z",
                            "format": "strict_date_optional_time"
                        }
                    }
                }
            ]
        }
    },
}

In [57]:
base_url = "https://116.101.122.180:5200/{index}*/_search"

headers = {
    "Authorization": "ApiKey",
    "Content-Type": "application/json"
    }

index_arr = ["kong-access-", "metricbeat-", "traces-apm", "apm-", "logs-apm", "metrics-apm"]

for index in tqdm(index_arr):
    url = base_url.format(index=index)
    data = {
        "from": 0,
        "size": 500,
        "query": queries[f"{index}"],
        "sort": [
            {
                "@timestamp": {
                    "order": "asc"
                }
            }
        ]
    }
    response = requests.get(
        url, headers=headers,
        json=data, verify=False
    )
    data = response.json()
    with open(f"./logs/{index}.json", "w") as json_file:
        json.dump(data, json_file, indent=4)

  0%|          | 0/6 [00:00<?, ?it/s]/home/aiteam/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1068: InsecureRequestWarning: Unverified HTTPS request is being made to host '116.101.122.180'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
 17%|█▋        | 1/6 [00:04<00:20,  4.13s/it]/home/aiteam/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1068: InsecureRequestWarning: Unverified HTTPS request is being made to host '116.101.122.180'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
 33%|███▎      | 2/6 [00:05<00:10,  2.72s/it]/home/aiteam/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1068: InsecureRequestWarning: Unverified HTTPS request is being made to host '116.101.122.180'. Adding certificate verification is str

In [ ]:
import pandas as pd
import json

# Load the JSON data from a file
with open('/home/aiteam/Documents/longvu02/example/logs/logs-apm-v2.json', 'r') as file:  # Replace with your actual file path
    data = json.load(file)

# Extracting relevant fields from the nested structure
extracted_data = []

for hit in data['hits']['hits']:
    source = hit['_source']
    error_info = source.get('error', {})
    stacktrace = error_info.get('stacktrace', [])
    
    # Prepare a dictionary for each record
    record = {
        'INFO message': source.get('message', ''),
        'WARD message': '',  # Assuming WARD message is not present in the provided structure
        'ERROR message': error_info.get('message', ''),
        'Stack trace': '\n'.join([f"{item['classname']} - {item['filename']}:{item['line']['number']} - {item['function']}" for item in stacktrace]),
        'Error code': error_info.get('type', ''),  # Assuming type as Error code
        'Error cause': error_info.get('exception', [{}])[0].get('message', '')  # Get first exception message as Error cause
    }
    
    extracted_data.append(record)

# Create a DataFrame from the extracted data
df = pd.DataFrame(extracted_data)

# Print the first 10 rows of the DataFrame
print(df.head(10))
df.to_csv('/home/aiteam/Documents/longvu02/example/csv/logs_apm.csv')

In [ ]:
import pandas as pd
import json

# Load the JSON data from a file
with open('/home/aiteam/Documents/longvu02/example/logs/logs-apm-v2.json', 'r') as file:  # Replace with your actual file path
    data = json.load(file)

# Extracting relevant fields from the nested structure
extracted_data = []

for hit in data['hits']['hits']:
    source = hit['_source']
    error_info = source.get('error', {})
    stacktrace = error_info.get('stacktrace', [])
    
    # Prepare a dictionary for each record
    record = {
        'INFO message': source.get('message', ''),
        'WARD message': '',  # Assuming WARD message is not present in the provided structure
        'ERROR message': error_info.get('message', ''),
        'Stack trace': '\n'.join([f"{item['classname']} - {item['filename']}:{item['line']['number']} - {item['function']}" for item in stacktrace]),
        'Error code': error_info.get('type', ''),  # Assuming type as Error code
        'Error cause': error_info.get('exception', [{}])[0].get('message', '')  # Get first exception message as Error cause
    }
    
    extracted_data.append(record)

# Create a DataFrame from the extracted data
df = pd.DataFrame(extracted_data)

# Print the first 10 rows of the DataFrame
print(df.head(10))
df.to_csv('/home/aiteam/Documents/longvu02/example/csv/metrics_apm.csv')

In [ ]:
import pandas as pd
import json

# Load the JSON data from the file
with open('/home/aiteam/Documents/longvu02/example/logs/traces-apm-v2.json', 'r') as file:  # Replace with your actual file path
    data = json.load(file)



# Prepare a list to hold the extracted records
extracted_data = []

# Iterate through each hit in the JSON data
for hit in data['hits']['hits']:
    source = hit['_source']
    
    # Extract relevant fields
    transaction = source.get('transaction', {})
    
    # Ensure duration is accessed correctly and converted to seconds
    duration_us = transaction.get('duration', {}).get('us', 0)  # Default to 0 if not found
    
    record = {
        'transactions': transaction.get('name', ''),
        'spans': source.get('span', {}).get('name', ''),
        'transaction start time': source.get('@timestamp', ''),
        'transaction end time': (int(transaction.get('@timestamp', 0)) + duration_us) / 1_000_000,  # Convert microseconds to seconds
        'trace path': source.get('trace', {}).get('id', '')
    }
    
    extracted_data.append(record)

# Create a DataFrame from the extracted data
df = pd.DataFrame(extracted_data)

# Print the first 10 rows of the DataFrame
print(df.head(10))
df.to_csv('/home/aiteam/Documents/longvu02/example/csv/traces_apm.csv')





